## Import libraries and dataset.

In [61]:
from collections import Counter
from sklearn.cluster import KMeans
from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image
import requests
import webcolors
import pandas as pd
pd.options.display.max_columns = 7

In [2]:
lea_card_data = pd.read_csv("C:\\Users\\chris\\Desktop\\lea_card_data.csv")

## Create subset with art/artist data.

In [62]:
df = pd.concat([lea_card_data.name_x, lea_card_data.types, lea_card_data.colorIdentity, lea_card_data.artist_x,
                lea_card_data.setCode, lea_card_data.released_at, lea_card_data.scryfallid, lea_card_data.artlink], axis=1)
df

,name_x,types,colorIdentity,...,released_at,scryfallid,artlink
0,Animate Wall,['Enchantment'],['W'],...,1993-08-05,d5c83259-9b90-47c2-b48e-c7d78519e792,https://c1.scryfall.com/file/scryfall-cards/ar...
1,Armageddon,['Sorcery'],['W'],...,1993-08-05,5b6ddce7-b9c5-431d-a0b0-46d4aa93cbcb,https://c1.scryfall.com/file/scryfall-cards/ar...
2,Balance,['Sorcery'],['W'],...,1993-08-05,6f9ea46a-411f-40ce-a873-a905180093f4,https://c1.scryfall.com/file/scryfall-cards/ar...
3,Benalish Hero,['Creature'],['W'],...,1993-08-05,11600105-56c6-4073-a4a6-8469030b39c9,https://c1.scryfall.com/file/scryfall-cards/ar...
4,Black Ward,['Enchantment'],['W'],...,1993-08-05,15967a39-303f-457d-bcde-51837c8d63e1,https://c1.scryfall.com/file/scryfall-cards/ar...
...,...,...,...,...,...,...,...
290,Swamp,['Land'],['B'],...,1993-08-05,edeba4e1-9012-4962-8797-3a1f6a660952,https://c1.scryfall.com/file/scryfall-cards/ar...
291,Mountain,['Land'],['R'],...,1993-08-05,eace2c85-976c-425e-9800-5a6ccbd91b56,https://c1.scryfall.com/file/scryfall-cards/ar...
292,Mountain,['Land'],['R'],...,1993-08-05,1a96315d-edc6-4d68-a260-d14f61281dc1,https://c1.scryfall.com/file/scryfall-cards/ar...
293,Forest,['Land'],['G'],...,1993-08-05,6f1c8cb0-38eb-408b-94e8-16db83999b3b,https://c1.scryfall.com/file/scryfall-cards/ar...


## Functions to return most similar color for a given hex code.

In [5]:
def get_approx_color(hex_color):
    orig = webcolors.hex_to_rgb(hex_color)
    similarity = {}
    for hex_code, color_name in webcolors.CSS3_HEX_TO_NAMES.items():
        approx = webcolors.hex_to_rgb(hex_code)
        similarity[color_name] = sum(np.subtract(orig, approx) ** 2)
    return min(similarity, key=similarity.get)

def get_color_name(hex_color):
    try:
        return webcolors.hex_to_name(hex_color)
    except ValueError:
        return get_approx_color(hex_color)

## Functions to return most prominent colors in an image.

In [46]:
def preprocess(raw):
    image = cv2.resize(raw, (900, 600), interpolation = cv2.INTER_AREA)                                          
    image = image.reshape(image.shape[0]*image.shape[1], 3)
    return image

def rgb_to_hex(rgb_color):
    hex_color = "#"
    for i in rgb_color:
        hex_color += ("{:02x}".format(int(i)))
    return hex_color

def analyze(img):
    mod_image = preprocess(img)
    
    clf = KMeans(n_clusters = 5)
    color_labels = clf.fit_predict(mod_image)
    center_colors = clf.cluster_centers_
    counts = Counter(color_labels)
    ordered_colors = [center_colors[i] for i in counts.keys()]
    hex_colors = [rgb_to_hex(ordered_colors[i]) for i in counts.keys()]

    #plt.figure(figsize = (12, 8))
    #plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)
    #plt.savefig("my_pie.png")
    
    total = 0
    for i in list(counts.values()):
        total += i
    tuple_list = []
    color_list = []
    for index,color in enumerate(hex_colors):
        tuple_list.append(tuple([get_color_name(color), color, round((list(counts.values())[index]/total)*100)]))
    tuple_list.sort(key = lambda x: x[2], reverse = True)
    for tup in tuple_list:
        for j in tup:
            color_list.append(j)
    #print(color_list)
    return color_list

## Grab and convert image, analyze, and add results to list. (~25 min/set)

In [50]:
url_list = df.artlink.tolist()
analysis_list = []
for url in url_list:
    im = Image.open(requests.get(url, stream=True).raw).convert('RGB')
    cv2_image = np.array(im)
    analysis_list.append(analyze(cv2_image))

## Use results to create dataframe.

In [63]:
df_analysis = pd.DataFrame(analysis_list, columns=["col_string1", "hex1", "col_prop1",
                                                   "col_string2", "hex2", "col_prop2",
                                                   "col_string3", "hex3", "col_prop3",
                                                   "col_string4", "hex4", "col_prop4",
                                                   "col_string5", "hex5", "col_prop5",])
df_analysis

,col_string1,hex1,col_prop1,...,col_string5,hex5,col_prop5
0,silver,#babbbb,25,...,darkslategray,#4b4951,14
1,dimgray,#7c676c,37,...,darkslategray,#332631,3
2,sienna,#9f7c3c,26,...,gray,#898282,14
3,rosybrown,#af9380,33,...,dimgray,#6e5a54,7
4,dimgray,#655353,31,...,dimgray,#786663,8
...,...,...,...,...,...,...,...
290,silver,#c6b9aa,31,...,dimgray,#736a5e,9
291,dimgray,#5d575a,32,...,silver,#c3c7be,3
292,cadetblue,#5e95ac,29,...,darkslategray,#24272b,9
293,teal,#1b6474,26,...,steelblue,#367b89,13


## Combine dataframes horizontally. Save and export to CSV.

In [52]:
merged_df = pd.concat([df, df_analysis], axis=1)

In [54]:
merged_df.to_csv("C:\\Users\\chris\\Desktop\\lea_color_data1.csv", index=False)